# 树回归

本章内容
* CART算法
+ 回归与模型树
- 树剪枝算法
+ Python中GUI的使用  

线性回归创建的模型需要拟合所有的样本点(局部加权线性回归除外)。\begin{problem}
当数据拥有众多特征并且特征之间关系十分复杂时，构建全局模型的想法就显得太难了，也很略显示笨拙。而且，实际生活中很多问题都是非线性的，不可能使用全局线性模型来拟合任何数据。
\end{problem}
\begin{proposition}
一种可行的方法是将数据集切分成很多份易建模的数据，然后利用线性回归技术来建模。如果首次切分后仍然难以拟合线性模型就继续切分。在这种切分方式下，树结构和回归法就相当有用。  
\end{proposition}

首先介绍一个新的叫做CART(Classification And Regression Trees，分类回归树)的树构建算法。**该算法既可以用于分类还可以用于回归**，因此非常值得学习。接着，利用CART算法构建回归树并介绍其中的树剪枝技术(防止树的过拟合）。之后引入了一个更高级的模型树算法。与回归树的做法(在每个叶节点上使用各自的均值做预测)不同，**该算法需要在每个叶节点上都构建出一个线性模型。**在这些树的构建算法中有一些需要调整的参数，所以还会介绍如何使用Python中的Tkinter模块建立图形交互界面。最后，在该界面的辅助下分析参数对回归效果的影响。

## 复杂数据的局部性建模

|树回归||
|:-|:-|
|优点|可以对复杂和非线性的数据建模|
|缺点|结果不易理解|
|适用数据类型|数值型和标称型数据|

决策树不断将数据切分成小数据集，直到所有目标变量完全相同，或者数据不能再切分为止。决策树是一种贪心算法，**它要在给定时间内做出最佳选择，但并不关心能否达到全局最优。**

**ID3的做法是每次选取当前最佳的特征来分割数据，并按照该特征的所有可能取值来切分。**也就是说，如果一个特征有4种取值，那么数据将被切成4份。一旦按某特征切分后，该特征在之后的算法执行过程中将不会再起作用，所以有观点**认为这种切分方式过于迅速**。另外一种方法是二元切分法，即每次把数据集切成两份。如果数据的某特征值等于切分所要求的值，那么这些数据就进入树的左子树，反之则进入树的右子树。

除了切分过于迅速外，ID3算法还存在另一个问题，**它不能直接处理连续型特征。**只有事先将连续型特征转换成离散型，才能在ID3算法中使用。但这种转换过程会破坏连续型变量的内在性质。**而使用二元切分法则易于对树构建过程进行调整以处理连续型特征。**具体做法：如果特征值大于给定值就走左子树，否则就走右子树。另外，二元切分法也节省了树的构建时间，但这点意义也不是特别大，因为这些树构建一般是**离线完成，时间并非需要重点关注的因素**。

**CART是十分著名且广泛记载的树构建算法，它使用二元切分来处理连续型变量**。对CART稍作修改就可以处理回归问题。如果选用其他方法来代替香农熵，就可以使用树构建算法来完成回归。

|树回归的一般方法||
|:-|:-|
|收集数据|采用任意方法收集数据|
|准备数据|需要数值型的数据,标称型数据应该映射成二值型数据|
|分析数据|绘出数据的二维可视化显示结果，以字典方式生成树|
|训练算法|大部分时间都花费在叶节点树模型的构建上|
|测试算法|使用测试数据上的R^{2}值来分析模型的效果|
|使用算法|使用训练出的树做预测，预测结果还可以用来做很多事情|

## 连续和离散型特征的树的构建

在树的构建过程中，需要解决多种类型数据的存储问题。使用一部字典来存储树的数据结构：
+ 待切分的特征
+ 待切分的特征值
+ 右子树。当不再需要切分的时候，也可以是单个值
+ 左子树。与右子树类似

**CART算法只做二元切分**，所以这里可以固定树的数据结构。树包含左键和右键，可以存储另一棵子树或者单个值。字典还包含特征和特征值这两个键，它们给出切分算法所有的特征和特征值。

In [1]:
# 当然，读者可以用面向对象的编程模式来建立这个数据结构。例如，可以用下面的Python代码来建立树节点：
class treeNode():
    def __init__(self, feat, val, right, left):
        featureToSplitOn = feat
        valueOfSplit = val
        rightBranch = right
        leftBranch = left

当使用C++这样不太灵活的编程语言时，你可能要用面向对象编程模式来实现树结构。**Python具有足够的灵活性，可以直接使用字典来存储树结构而无须另外自定义一个类，从而有效地减少代码量。**Python不是一种强类型编程语言，因此接下来会看到，树的每个分枝还可以再包含其他树、数值型数据甚至是向量。

函数**createTree()的伪代码**大致如下：  
　找到最佳的待切分特征:  
　　如果该节点不能再分，将该节点存为叶节点  
　　执行二元切分  
　　在右子树调用createTree()方法  
　　在左子树调用createTree()方法

In [5]:
# CART算法的实现代码
import numpy as np


def loadDataSet(fileName):  # general function to parse tab -delimited floats
    dataMat = []  # assume last column is target value
    fr = open(fileName)
    for line in fr.readlines():
        curLine = line.strip().split('\t')
        fltLine = list(map(float, curLine))  # map all elements to float()
        dataMat.append(fltLine)
    return dataMat


def binSplitDataSet(dataSet, feature, value):
    mat0 = dataSet[np.nonzero(dataSet[:, feature] > value)[0], :]  # [0]源代码有这个，但这样只取了一个数据，错了
    mat1 = dataSet[np.nonzero(dataSet[:, feature] <= value)[0], :]  # [0]
    return mat0, mat1


# assume dataSet is NumPy Mat so we can array filtering
def createTree(dataSet, leafType=regLeaf, errType=regErr, ops=(1, 4)):
    feat, val = chooseBestSplit(dataSet, leafType, errType, ops)  # choose the best split
    if feat == None:
        return val  # if the splitting hit a stop condition return val
    retTree = {}
    retTree['spInd'] = feat
    retTree['spVal'] = val
    lSet, rSet = binSplitDataSet(dataSet, feat, val)
    retTree['left'] = createTree(lSet, leafType, errType, ops)
    retTree['right'] = createTree(rSet, leafType, errType, ops)
    return retTree

loadDataSet()，该函数读取一个以tab键为分隔符的文件，然后将每行的内容保存成一组浮点数。

binSplitDataSet()，该函数有3个参数：数据集合、待切分的特征和该特征的某个值。在给定特征和特征值的情况下，**该函数通过数组过滤方式将上述数据集合切分得到两个子集并返回。** 

createTree()，它有4个参数：数据集和其他3个可选参数。这些可选参数决定了树的类型：**leafType给出建立叶节点的函数；errType代表误差计算函数；而ops是一个包含树构建所需其他参数的元组。**该函数首先尝试将数据集分成两个部分，切分由函数chooseBestSplit()完成。如果满足停止条件，chooseBestSplit()将返回None和某类模型的值。如果构建的是回归树，该模型是一个常数。如果是模型树，其模型是一个线性方程。

In [3]:
testMat = np.mat(np.eye(4))
mat0, mat1 = binSplitDataSet(testMat, 1, 0.5)
mat0
mat1

matrix([[0., 1., 0., 0.]])

matrix([[1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])

## 将CART算法用于回归

\begin{problem}
要对数据的复杂关系建模，我们已经决定借用树结构来帮助切分数据，那么如何实现数据的切分呢？怎么才能知道是否已经充分切分呢？
\end{problem}
\begin{proposition}
取决于叶节点的建模方式。回归树假设叶节点是常数值，这种策略认为数据中的复杂关系可以用树结构来概括。
\end{proposition}

**为成功构建以分段常数为叶节点的树，需要度量出数据的一致性。**

\begin{problem}
离散数据混乱度：熵，基尼不纯度。那么如何计算连续型数值的混乱度呢？
\end{problem}
\begin{proposition}
事实上，在数据集上计算混乱度是非常简单的。首先计算所有数据的均值，然后计算每条数据的值到均值的差值。为了对正负差值同等看待，一般使用绝对值或平方值来代替上述差值。上述做法有点类似于前面介绍过的统计学中常用的方差计算。唯一的不同就是，方差是平方误差的均值(均方差)，而这里需要的是平方误差的总值(总方差)。**总方差**可以通过均方差乘以数据集中样本点的个数来得到。
\end{proposition}

### 构建树

chooseBestSplit()函数。给定某个误差计算方法，该函数会找到数据集上最佳的二元切分方式。另外，该函数还要确定什么时候停止切分，一旦停止切分会生成一个叶节点。因此只需完成两件事：**用最佳方式切分数据集和生成相应的叶节点。**

**leafType是对创建叶节点的函数的引用，errType是对前面介绍的总方差计算函数的引用，而ops是一个用户定义的参数构成的元组，用以完成树的构建**。  
函数的**目标是找到数据集切分的最佳位置。它遍历所有的特征及其可能的取值来找到使误差最小化的切分阈值**。

**伪代码**大致如下：  
　对每个特征：  
　　对每个特征值：  
　　　将数据集切分成两份  
　　　计算切分的误差  
　　　如果当前误差小于当前最小误差，那么将当前切分设定为最佳切分并更新最小误差  
　返回最佳切分的特征和阈值

In [13]:
def regLeaf(dataSet):  # returns the value used for each leaf
    return np.mean(dataSet[:, -1])


def regErr(dataSet):
    return np.var(dataSet[:, -1]) * np.shape(dataSet)[0]


def chooseBestSplit(dataSet, leafType=regLeaf, errType=regErr, ops=(1, 4)):
    tolS = ops[0]
    tolN = ops[1]
    # if all the target variables are the same value: quit and return value
    if len(set(dataSet[:, -1].T.tolist()[0])) == 1:  # exit cond 1
        return None, leafType(dataSet)
    m, n = np.shape(dataSet)
    # the choice of the best feature is driven by Reduction in RSS error from mean
    S = errType(dataSet)
    bestS = np.inf
    bestIndex = 0
    bestValue = 0
    for featIndex in range(n-1):
        for splitVal in set(dataSet[:, featIndex].T.tolist()[0]):
            mat0, mat1 = binSplitDataSet(dataSet, featIndex, splitVal)
            if (np.shape(mat0)[0] < tolN) or (np.shape(mat1)[0] < tolN):
                continue
            newS = errType(mat0) + errType(mat1)
            if newS < bestS:
                bestIndex = featIndex
                bestValue = splitVal
                bestS = newS
    # if the decrease (S-bestS) is less than a threshold don't do the split
    if (S - bestS) < tolS:
        return None, leafType(dataSet)  # exit cond 2
    mat0, mat1 = binSplitDataSet(dataSet, bestIndex, bestValue)
    if (np.shape(mat0)[0] < tolN) or (np.shape(mat1)[0] < tolN):  # exit cond 3
        return None, leafType(dataSet)
    return bestIndex, bestValue  # returns the best feature to split on
    # and the value used for that split

**regLeaf()，它负责生成叶节点。**当chooseBestSplit()函数确定不再对数据进行切分时，将调用该regLeaf()函数来得到叶节点的模型。在回归树中，该模型其实就是目标变量的均值。

**误差估计函数regErr()**。该函数在给定数据上计算目标变量的平方误差。

chooseBestSplit()，**目的是找到数据的最佳二元切分方式，最佳切分也就是使得切分后能达到最低误差的切分。**。如果找不到一个“好”的二元切分，该函数返回None并同时调用regLeaf()方法来产生叶节点。如果找到了一个“好”的切分方式，则返回特征编号和切分特征值。三种情况不切分：❶不同剩余特征值的数目为1 ❷切分数据集后效果提升不够大 ❸检查两个切分后的子集大小，如果某个子集的大小小于用户定义的参数tolN  
在所有可能的特征及其可能取值上遍历，找到最佳的切分方式，如果这些提前终止条件都不满足，那么就返回切分特征和特征值

chooseBestSplit()一开始为ops设定了tolS和tolN这两个值。它们是用户指定的参数，用于控制函数的停止时机。其中变量tolS是容许的误差下降值，tolN是切分的最少样本数。

### 运行代码

In [15]:
myDat = loadDataSet('ex00.txt')
myMat = np.mat(myDat)
createTree(myMat)

{'spInd': 0,
 'spVal': 0.48813,
 'left': 1.0180967672413792,
 'right': -0.04465028571428572}

In [9]:
a=[[1],[2],[3]];
set(a)

TypeError: unhashable type: 'list'